<a href="https://colab.research.google.com/github/nonohuang0819/kaggle/blob/main/task2/task2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
from google.colab import drive
drive.mount("/content/drive")

import pandas as pd
import sklearn

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
filepath = "/content/drive/MyDrive/113-1機器學習概論/KaggleContest/kaggle/task2/introml_2024_task2_train.csv"
df = pd.read_csv(filepath)
df

,f0,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13,f14,f15,f16,f17,class
0,f00,f11,f20,f30,f40,f50,f61,f71,f81,0.3839,1.5223,0.5516,0.0579,2.2677,1.1361,0.9204,1.8718,0.7433,C0
1,f00,f10,f20,f30,f40,f50,f61,f70,f81,1.4089,1.7259,0.9097,1.5688,0.9065,1.2314,0.9521,1.5515,0.7560,C0
2,f01,f10,f20,f30,f41,f50,f60,f71,f80,0.2752,0.2936,0.5023,1.6234,1.7168,1.3118,0.3520,2.5584,2.0897,C0
3,f01,f11,f20,f31,f41,f50,f60,f70,f80,0.1777,0.1441,2.1431,1.3822,2.1976,0.7040,0.1160,0.1408,2.0033,C0
4,f00,f11,f20,f30,f40,f50,f60,f71,f80,0.1692,0.2393,2.5039,0.8708,1.6206,0.6319,0.6632,0.6857,0.6148,C0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3995,f01,f10,f20,f30,f41,f50,f61,f71,f80,0.3347,0.7849,0.0796,0.7342,0.5169,0.7413,0.4696,0.6958,0.6273,C4
3996,f00,f10,f20,f30,f41,f50,f61,f70,f80,0.2989,0.6772,0.0325,2.4367,0.2669,1.1615,0.4936,0.3491,0.8280,C4
3997,f00,f10,f20,f31,f41,f50,f61,f71,f80,0.5245,0.6542,1.0248,1.4574,0.1816,0.5517,0.1486,0.3595,1.6582,C4
3998,f00,f10,f20,f30,f41,f50,f61,f70,f80,0.7995,0.2713,0.4453,1.1236,1.0424,0.4081,1.0728,0.5401,1.0678,C4
